# Step 1: Install required libraries

In [ ]:
!pip install transformers datasets torch

# Step 2: Upload dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

# Step 3: Load and preprocess dataset

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

dataset = load_dataset('json', data_files='tactics_dataset.json')

tokenizer = AutoTokenizer.from_pretrained("gpt2")

def tokenize_data(sample):
    fen_tokens = tokenizer(sample['fen'], truncation=True, padding="max_length", max_length=512)
    move_tokens = tokenizer(sample['moves'], truncation=True, padding="max_length", max_length=128)
    return {
        'input_ids': fen_tokens['input_ids'] + move_tokens['input_ids'],
        'attention_mask': fen_tokens['attention_mask'] + move_tokens['attention_mask'],
        'labels': tokenizer(sample['tactic'], truncation=True, padding="max_length", max_length=5)['input_ids']
    }

tokenized_dataset = dataset.map(tokenize_data, batched=True)

# Step 4: Load pre-trained model

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Step 5: Define training arguments

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',         
    num_train_epochs=3,             
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',           
    logging_steps=10,               
    evaluation_strategy="steps"     
)

# Step 6: Fine-tune the model

In [ ]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=tokenized_dataset['train'],         
    eval_dataset=tokenized_dataset['test']             
)

trainer.train()

# Step 7: Evaluate the model

In [ ]:
results = trainer.evaluate()
print(results)

# Step 8: Save the model

In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')
!zip -r fine_tuned_model.zip ./fine_tuned_model
files.download('fine_tuned_model.zip')


# Step 9: Generate tactics

In [ ]:
def generate_tactic(fen):
    inputs = tokenizer(fen, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=20)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

fen_example = "r1bqkbnr/pppppppp/n7/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
tactic = generate_tactic(fen_example)
print(tactic)